# Install Dependencies

In [ ]:
!pip install -U "openmim" &> /dev/null
!mim install "mmengine" &> /dev/null
!mim install "mmcv==2.1.0" &> /dev/null
!mim install "mmdet>=3.1.0" &> /dev/null
!pip install "gdown==4.6.0" &> /dev/null

# Install MMPose

In [ ]:
%cd "/content"
!rm -rf "/content/mmpose"
!git clone "https://github.com/open-mmlab/mmpose.git"
%cd "/content/mmpose"
!pip install -r "requirements.txt" &> /dev/null
!pip install -v -e . &> /dev/null

/content
Cloning into 'mmpose'...
remote: Enumerating objects: 31259, done.
remote: Counting objects: 100% (361/361), done.
remote: Compressing objects: 100% (241/241), done.
remote: Total 31259 (delta 172), reused 263 (delta 118), pack-reused 30898
Receiving objects: 100% (31259/31259), 53.49 MiB | 28.95 MiB/s, done.
Resolving deltas: 100% (22000/22000), done.
/content/mmpose


In [ ]:
%cd "/content/mmpose"

/content/mmpose


# Setup Dataset

Download Dataset files from google drive

In [ ]:
!gdown "1UT4mwkyN8cnUN6AQo2aday8-fYqhsu0D"

Downloading...
From: https://drive.google.com/uc?id=1UT4mwkyN8cnUN6AQo2aday8-fYqhsu0D
To: /content/mmpose/RightLeftCombined_Cropped.zip
100% 8.21G/8.21G [01:50<00:00, 74.4MB/s]


Unzip downloaded files nito `dataset` folder

In [ ]:
!rm -rf "dataset"
!mkdir "dataset"
!unzip -oj "RightLeftCombined_Cropped.zip" -d "dataset" &> /dev/null
!rm -rf "__MACOSX"

'/content/mmpose/dataset/cropped_screenshot_20240722_113530.png'

'/content/mmpose/dataset/cropped_screenshot_20240722_113530.png'

In [ ]:
!rm

rm: missing operand
Try 'rm --help' for more information.


In [ ]:
!ls -l "dataset" | wc -l

5017


# Import Libraries

In [ ]:
!pip install requests==2.28.2

In [ ]:
%cd "/content/mmpose"

import gc
import os
import time
import shutil
import logging

from tqdm import tqdm

import cv2
import mmcv
import mmengine
import numpy as np
import json_tricks as json

import torch
from mmpose.apis import inference_topdown
from mmpose.apis import init_model as init_pose_estimator
from mmpose.evaluation.functional import nms
from mmpose.registry import VISUALIZERS
from mmpose.structures import merge_data_samples, split_instances
from mmpose.utils import adapt_mmdet_pipeline

try:
    from mmdet.apis import inference_detector, init_detector
    has_mmdet = True
except (ImportError, ModuleNotFoundError):
    has_mmdet = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)


/content/mmpose
device: cuda


Function to process one image using pose estimation model

In [ ]:
def process_one_image(args,
                      img,
                      detector,
                      pose_estimator,
                      visualizer=None,
                      show_interval=0):
    """Visualize predicted keypoints (and heatmaps) of one image."""

    # predict bbox
    det_result = inference_detector(detector, img)
    pred_instance = det_result.pred_instances.cpu().numpy()
    bboxes = np.concatenate((pred_instance.bboxes, pred_instance.scores[:, None]), axis=1)
    bboxes = bboxes[np.logical_and(pred_instance.labels == args['det_cat_id'],
                                   pred_instance.scores >= args['bbox_thr'])]
    bboxes = bboxes[nms(bboxes, args['nms_thr']), :4]

    # predict keypoints
    pose_results = inference_topdown(pose_estimator, img, bboxes)
    data_samples = merge_data_samples(pose_results)

    # show the results
    if isinstance(img, str):
        img = mmcv.imread(img, channel_order='rgb')
    elif isinstance(img, np.ndarray):
        img = mmcv.bgr2rgb(img)

    if visualizer is not None:
        visualizer.add_datasample(
            'result',
            img,
            data_sample=data_samples,
            draw_gt=False,
            draw_heatmap=args['draw_heatmap'],
            draw_bbox=args['draw_bbox'],
            show_kpt_idx=args['show_kpt_idx'],
            skeleton_style=args['skeleton_style'],
            show=False,
            wait_time=show_interval,
            kpt_thr=args['kpt_thr'])
    return data_samples.get('pred_instances', None)


Define configurations required for pose estimation pipeline

In [ ]:
args = dict(
    # 'Config file for detection'
    det_config = "demo/mmdetection_cfg/rtmdet_nano_320-8xb32_hand.py",
    # 'Checkpoint file for detection'
    det_checkpoint = "https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmdet_nano_8xb32-300e_hand-267f9c8f.pth",
    # 'Config file for pose'
    pose_config = "configs/hand_2d_keypoint/rtmpose/hand5/rtmpose-m_8xb256-210e_hand5-256x256.py",
    # 'Checkpoint file for pose'
    pose_checkpoint = "https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmpose-m_simcc-hand5_pt-aic-coco_210e-256x256-74fb594_20230320.pth",
    # 'root of the output img file. ' 'Default not saving the visualization images.'
    output_root = 'vis_results/',
    # 'whether to save predicted results'
    save_predictions = True,
    # 'Category id for bounding box detection model'
    det_cat_id = 0,
    # 'Bounding box score threshold'
    bbox_thr = 0.3,
    # 'IoU threshold for bounding box NMS'
    nms_thr = 0.3,
    # 'Visualizing keypoint thresholds'
    kpt_thr = 0.3,
    # 'Draw heatmap predicted by the model'
    draw_heatmap = True,
    # 'Whether to show the index of keypoints'
    show_kpt_idx = True,
    # 'Skeleton style selection'; choices=['mmpose', 'openpose']
    skeleton_style = 'mmpose',
    # 'Keypoint radius for visualization'
    radius = 3,
    # 'Link thickness for visualization'
    thickness = 1,
    # 'The transparency of bboxes'
    alpha = 0.8,
    # 'Draw bboxes of instances'
    draw_bbox = True,
)


Main function that takes path of images directory and runs pose estimation model on all images in that directory one by one.

In [ ]:
def main(images_dir):
    images_name = os.listdir(images_dir)
    images_name = list(filter(lambda x: not x.startswith('.'), images_name))
    print('[images_name]', len(images_name))
    mmengine.mkdir_or_exist(args['output_root'])

    # build detector
    detector = init_detector(args['det_config'], args['det_checkpoint'], device=device)
    detector.cfg = adapt_mmdet_pipeline(detector.cfg)

    # build pose estimator
    pose_estimator = init_pose_estimator(
        args['pose_config'],
        args['pose_checkpoint'],
        device=device,
        cfg_options=dict(model=dict(test_cfg=dict(output_heatmaps=args['draw_heatmap']))))

    # build visualizer
    pose_estimator.cfg.visualizer.radius = args['radius']
    pose_estimator.cfg.visualizer.alpha = args['alpha']
    pose_estimator.cfg.visualizer.line_width = args['thickness']
    visualizer = VISUALIZERS.build(pose_estimator.cfg.visualizer)
    # the dataset_meta is loaded from the checkpoint and
    # then pass to the model in init_pose_estimator
    visualizer.set_dataset_meta(pose_estimator.dataset_meta, skeleton_style=args['skeleton_style'])

    for image_name in tqdm(images_name, total=len(images_name)):
        args['input'] = os.path.join(images_dir, image_name)
        output_file = os.path.join(args['output_root'], os.path.basename(args['input']))
        args['pred_save_path'] = f"{args['output_root']}/results_{os.path.splitext(os.path.basename(args['input']))[0]}.json"

        # inference
        pred_instances = process_one_image(args, args['input'], detector, pose_estimator, visualizer)
        pred_instances_list = split_instances(pred_instances)
        img_vis = visualizer.get_image()
        mmcv.imwrite(mmcv.rgb2bgr(img_vis), output_file)

        with open(args['pred_save_path'], 'w') as f:
            obj2dump = dict(meta_info=pose_estimator.dataset_meta, instance_info=pred_instances_list)
            json.dump(obj2dump, f, indent='\t')

        gc.collect()
        torch.cuda.empty_cache()


# Execute MMPose for Generating Annotations

Running pose estimation on dataset directory

In [ ]:
main('/content/mmpose/dataset')

[images_name] 5016
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmdet_nano_8xb32-300e_hand-267f9c8f.pth


Downloading: "https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmdet_nano_8xb32-300e_hand-267f9c8f.pth" to /root/.cache/torch/hub/checkpoints/rtmdet_nano_8xb32-300e_hand-267f9c8f.pth
/usr/local/lib/python3.10/dist-packages/mmdet/apis/inference.py:108: UserWarning: palette does not exist, random is used by default. You can also set the palette to customize.
  warnings.warn(


Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmpose-m_simcc-hand5_pt-aic-coco_210e-256x256-74fb594_20230320.pth


Downloading: "https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmpose-m_simcc-hand5_pt-aic-coco_210e-256x256-74fb594_20230320.pth" to /root/.cache/torch/hub/checkpoints/rtmpose-m_simcc-hand5_pt-aic-coco_210e-256x256-74fb594_20230320.pth
  0%|          | 0/5016 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/content/mmpose/mmpose/models/heads/coord_cls_heads/rtmcc_head.py:217: UserWarning: The predicted simcc values are normalized for visualization. This may cause discrepancy between the keypoint scores and the 1D heatmaps.
  warnings.warn('The predicted simcc values are normalized for '
/usr/local/lib/python3.10/dist-packages/json_tricks/encoders.py:419: UserWarning: json-tricks: numpy scalar seria

# Save the generated outputs

Zip the directory, containing json outputs and image outputs, for downloading or saving in drive

In [ ]:
!zip -r 'vis_results.zip' '/content/mmpose/vis_results/' &> /dev/null

Copy only the json output files in another directory to filter the output images

In [ ]:
os.makedirs('/content/mmpose/mmpose_outputs_json', exist_ok=True)
json_outputs = list(filter(lambda x: x.endswith('.json'), os.listdir('/content/mmpose/vis_results')))
for filename in json_outputs:
    src_filepath = os.path.join('/content/mmpose/vis_results', filename)
    dst_filepath = os.path.join('/content/mmpose/mmpose_outputs_json', filename)
    shutil.copy(src_filepath, dst_filepath)


Zip the directory, containing only json outputs, for downloading or saving in drive

In [ ]:
!zip -r 'mmpose_outputs_json.zip' '/content/mmpose/mmpose_outputs_json/' &> /dev/null

Copy the zip file in drive to use in super-gradients training script

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r '/content/mmpose/mmpose_outputs_json.zip' '/content/drive/MyDrive/python/ai-pointer'

In [ ]:
from google.colab import files

files.download('vis_results.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---